In [50]:
#Import required packages
import configurations as config
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
SPOTIFY_APP_NAME = config.SPOTIFY_APP_NAME
SPOTIPY_CLIENT_ID = config.SPOTIFY_CLIENT_ID
SPOTIPY_CLIENT_SECRET = config.SPOTIFY_CLIENT_SECRET
spotify_top_50_songs = f"https://open.spotify.com/playlist/{config.SPOTIFY_TOP_50_SONGS_URL}?si=1333723a6eff4b7f" 

In [28]:
def authenticate_spotify_api(client_id,client_secret):
    """
    This function is used to authenticate sptify API
    Args : 
    client_id : Obtained from the Spotify App 
    client_secret : Obtained from the Spotify App 
    Output:
    sp : spotify client used to interact with the API
    """
    try:
        client_credentials_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        return sp
    except Exception as e :
        print(f"{SPOTIFY_APP_NAME} : has resulted in the following error {e}")

In [47]:
def main ():
    try:
        sp = authenticate_spotify_api(client_id = SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET)
        top_50_songs_data = sp.playlist_tracks(spotify_top_50_songs)
    except Exception as e : 
        print(f"{SPOTIFY_APP_NAME} Error: {e}")

    # Fetch Album Data 
    album_list = []
    for album in top_50_songs_data['items']: 
        album_id = album['track']['album']['id']
        album_name = album['track']['album']['name']
        album_release_date = album['track']['album']['release_date']
        album_total_tracks = album['track']['album']['total_tracks']
        album_url = album['track']['album']['external_urls']['spotify']
        album_dict = {'album_id':album_id,'name':album_name,'release_date':album_release_date,'total_tracks':album_total_tracks
                    ,'url':album_url}
        album_list.append(album_dict)

    album_data = pd.DataFrame.from_dict(album_list)
    album_data['release_date'] = pd.to_datetime(album_data['release_date'])
    album_data.drop_duplicates(subset = 'album_id',inplace=True)

    # Fetch Artist Data 

    artist_list = []
    for artist_l1 in top_50_songs_data['items']:
        for key,value in artist_l1.items():
            if key == "track":
                for artist in value['artists']:
                    artist_dict = {'artist_id':artist['id'],'artist_name':artist['name'],'external_url':artist['href']}
                    artist_list.append(artist_dict)

    artist_data = pd.DataFrame.from_dict(artist_list)
    artist_data.drop_duplicates(subset='artist_id',inplace = True)

    # Fetch Songs Data 
    songs_list = []
    for songs in top_50_songs_data['items']:
        songs_dict = {'song_id':songs['track']['id'],'song_name':songs['track']['name'],'song_duration':songs['track']['duration_ms'],
                    'song_url':songs['track']['external_urls']['spotify'],'song_popularity':songs['track']['popularity'],
                    'album_id':songs['track']['album']['id'],
                    'artist_id':songs['track']['album']['artists'][0]['id']}
        songs_list.append(songs_dict)
        
    songs_data = pd.DataFrame.from_dict(songs_list)
    return album_data,artist_data,songs_data

if __name__ == "__main__":
    main()